### Regression Model using Latitude, Longitude, Sale Year, and Sale Price

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


df = pd.read_csv(r"C:\Users\conne\OneDrive\Desktop\Project 4\Project4-MachineLearning\df_joined_data.csv")

C:\Users\conne\AppData\Local\Temp\ipykernel_16656\3459330737.py:10: DtypeWarning: Columns (79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\conne\OneDrive\Desktop\Project 4\Project4-MachineLearning\df_joined_data.csv")


In [11]:
# Define the sale price range and desired property classes
price_lower_bound = 200000
price_upper_bound = 750000
desired_classes = [202, 203, 204, 205, 206, 207, 208, 209, 210, 234, 278, 295]

# Assuming the DataFrame is named 'data'
price_filtered_data = df[(df['Sale Price'] >= price_lower_bound) & (df['Sale Price'] <= price_upper_bound)]

# Filter the already price-filtered data for desired property classes
final_filtered_data_2 = price_filtered_data[price_filtered_data['Property Class'].isin(desired_classes)]

# Check the unique property classes and sale price stats to confirm the filter
print("Unique Property Classes:", final_filtered_data_2['Property Class'].unique())
print("Sale Price Stats:", final_filtered_data_2['Sale Price'].describe())

final_filtered_data_2.head(10)

Unique Property Classes: [207 206 278 295 205 203 210 204 202 234 209 208]
Sale Price Stats: count     28569.000000
mean     354461.566803
std      136040.772332
min      200000.000000
25%      247500.000000
50%      313750.000000
75%      425000.000000
max      750000.000000
Name: Sale Price, dtype: float64


,Unnamed: 0,pin,Property Class,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,...,lake_michigan_data_year,nearest_major_road_dist_ft,nearest_metra_stop_dist_ft,nearest_park_dist_ft,nearest_railroad_dist_ft,nearest_water_name,nearest_water_dist_ft,nearest_neighbor_1_dist_ft,nearest_neighbor_2_dist_ft,nearest_neighbor_3_dist_ft
1,1,20143100350000,207,70,2790.0,70,2.0,0.0,1.0,1.0,...,2022.0,4188.862414,2896.061077,886.525144,1745.812282,NaN,2559.083259,43.383443,76.822828,82.441660
40,40,20023170210000,206,10,9850.0,70,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.985531,52.500510,99.971062
43,43,26311020210000,278,280,5000.0,70,2.0,0.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.992272,79.986002,119.988149
49,49,20114270340000,295,20,1534.0,70,2.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.661547,19.678240,39.325516
56,56,20271200100000,205,30,6037.0,70,2.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.246278,35.498640,64.242746
81,81,25032040250000,203,120,4166.0,70,1.0,0.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,8619.076221,33.332655,33.334992,62.490991
101,101,20252260100000,205,83,5625.0,70,2.0,0.0,2.0,1.0,...,2017.0,NaN,2343.226246,NaN,NaN,Lk Michigan,3295.575031,39.042397,47.495242,84.988992
134,134,20102100110000,210,10,3750.0,70,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.489530,28.492459,52.729107
168,168,26172190490000,278,240,5886.0,70,2.0,0.0,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,Lk Michigan,4201.246970,46.878933,53.951436,75.239305
170,170,20112100330000,295,20,2104.0,70,2.0,0.0,3.0,2.0,...,2019.0,NaN,1699.005607,NaN,NaN,Lk Michigan,1723.186360,24.964526,47.187006,72.217299


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Convert 'Sale Date' to datetime and extract the year
final_filtered_data_2 = final_filtered_data_2.copy()
final_filtered_data_2['Sale Date'] = pd.to_datetime(final_filtered_data_2['Sale Date']).dt.year

# Prepare features and target
X = final_filtered_data_2[['latitude', 'longitude', 'Sale Date']]  # Ensure the correct columns are used
y = final_filtered_data_2['Sale Price']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the numeric transformer with an imputer and scaler
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute with the median value of each column
    ('scaler', StandardScaler())])

# Combine transformers for numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ['latitude', 'longitude', 'Sale Date'])])

# Create a full pipeline with preprocessor and estimator
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestRegressor(n_estimators=100, random_state=42))])

# Train the model
clf.fit(X_train, y_train)

# Evaluate the model
predictions = clf.predict(X_test)
rmse = mean_squared_error(y_test, predictions, squared=False)
mae = mean_absolute_error(y_test, predictions)

print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")

Root Mean Squared Error: 97432.48678828367
Mean Absolute Error: 70742.08469768404


In [15]:
# Create DataFrame to compare predictions with actual values
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop=True)

# Display the first 10 entries of the results DataFrame
print(results.head(10))

   Prediction  Actual
0   291515.00  210000
1   383540.00  395000
2   213048.00  205000
3   221740.00  206000
4   230789.00  218000
5   450875.00  275000
6   269368.00  235000
7   228414.85  214000
8   358235.00  450000
9   285290.00  212000
